## Get data

In [20]:
# %load ../src/data/get_data.py
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''
    git_pull = subprocess.Popen( "git pull" ,
                         cwd = os.path.dirname( '../data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


if __name__ == '__main__':
    get_johns_hopkins()

Error : b''
out : b'Already up to date.\n'


## Process data

In [21]:
# %load ../src/data/process_JH_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set
    '''

    data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))

if __name__ == '__main__':

    store_relational_JH_data()

 Number of rows stored: 261630


## Filter and doubling Rate Calculation

In [22]:
# %load ../src/features/build_features1.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate
        Parameters:
        ----------
        in_array : pandas.series
        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=11):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)
        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result
        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'
        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame
        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame
        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())

the test slope is: [2.]
             date state  country   confirmed  confirmed_filtered  \
141367 2022-07-23    no  Germany  30331133.0        3.033446e+07   
141368 2022-07-24    no  Germany  30331133.0        3.042504e+07   
141369 2022-07-25    no  Germany  30476605.0        3.051562e+07   
141370 2022-07-26    no  Germany  30598385.0        3.060620e+07   
141371 2022-07-27    no  Germany  30702511.0        3.069678e+07   

        confirmed_DR  confirmed_filtered_DR  
141367  6.586269e+02             331.698142  
141368  3.033113e+07             334.888710  
141369  4.176697e+02             335.888710  
141370  2.280148e+02             336.888710  
141371  2.708427e+02             337.888710  


## Visualization

In [39]:
# %load ../src/visualization/visualize1.py
import pandas as pd
import numpy as np

import dash
dash.__version__
from dash import dcc
from dash import html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = "browser"

import os
print(os.getcwd())
df_input_large=pd.read_csv('../data/processed/COVID_final_set.csv',sep=';')


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data
    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.
    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['Australia', 'Germany','Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]
        print(df_plot)
        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

C:\Users\demb011\Enterprise_Data_Science\Enterprise_Data_Science\notebooks
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on htt

              date state  country   confirmed  confirmed_filtered  \
41      2020-01-22    no  Germany         0.0       -1.454545e+00   
323     2020-01-23    no  Germany         0.0       -6.909091e-01   
609     2020-01-24    no  Germany         0.0        7.272727e-02   
894     2020-01-25    no  Germany         0.0        8.363636e-01   
1180    2020-01-26    no  Germany         0.0        1.600000e+00   
...            ...   ...      ...         ...                 ...   
260245  2022-07-23    no  Germany  30331133.0        3.033446e+07   
260530  2022-07-24    no  Germany  30331133.0        3.042504e+07   
260815  2022-07-25    no  Germany  30476605.0        3.051562e+07   
261100  2022-07-26    no  Germany  30598385.0        3.060620e+07   
261384  2022-07-27    no  Germany  30702511.0        3.069678e+07   

        confirmed_DR  confirmed_filtered_DR  
41               NaN                    NaN  
323              NaN                    NaN  
609              NaN             

              date state  country   confirmed  confirmed_filtered  \
41      2020-01-22    no  Germany         0.0       -1.454545e+00   
323     2020-01-23    no  Germany         0.0       -6.909091e-01   
609     2020-01-24    no  Germany         0.0        7.272727e-02   
894     2020-01-25    no  Germany         0.0        8.363636e-01   
1180    2020-01-26    no  Germany         0.0        1.600000e+00   
...            ...   ...      ...         ...                 ...   
260245  2022-07-23    no  Germany  30331133.0        3.033446e+07   
260530  2022-07-24    no  Germany  30331133.0        3.042504e+07   
260815  2022-07-25    no  Germany  30476605.0        3.051562e+07   
261100  2022-07-26    no  Germany  30598385.0        3.060620e+07   
261384  2022-07-27    no  Germany  30702511.0        3.069678e+07   

        confirmed_DR  confirmed_filtered_DR  
41               NaN                    NaN  
323              NaN                    NaN  
609              NaN             

              date state  country   confirmed  confirmed_filtered  \
41      2020-01-22    no  Germany         0.0       -1.454545e+00   
323     2020-01-23    no  Germany         0.0       -6.909091e-01   
609     2020-01-24    no  Germany         0.0        7.272727e-02   
894     2020-01-25    no  Germany         0.0        8.363636e-01   
1180    2020-01-26    no  Germany         0.0        1.600000e+00   
...            ...   ...      ...         ...                 ...   
260245  2022-07-23    no  Germany  30331133.0        3.033446e+07   
260530  2022-07-24    no  Germany  30331133.0        3.042504e+07   
260815  2022-07-25    no  Germany  30476605.0        3.051562e+07   
261100  2022-07-26    no  Germany  30598385.0        3.060620e+07   
261384  2022-07-27    no  Germany  30702511.0        3.069678e+07   

        confirmed_DR  confirmed_filtered_DR  
41               NaN                    NaN  
323              NaN                    NaN  
609              NaN             

## SIR Model

In [ ]:
# %load ../src/visualization/visSIR.py
import pandas as pd
import numpy as np

from scipy import optimize
from scipy import integrate

import dash
dash.__version__
from dash import dcc
from dash import html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = "browser"

import os
print(os.getcwd())

df_analyse = pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';')  
df_analyse = df_analyse.sort_values('date',ascending=True)
new_Frame = df_analyse.drop('date',axis=1)
data_top = new_Frame.columns


beta = 0.4
gamma = 0.1
R0 = 0
N0 = 10000000

i=50
n=100

def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt


ydata1=np.array(new_Frame['Germany'][i:n])
t = np.arange(len(ydata1))
N0 = 80000000
I0 = ydata1[0]
S0 = N0-I0
popt, pcov = optimize.curve_fit(fit_odeint, t, ydata1) ## Train the model / Fit the model
perr = np.sqrt(np.diag(pcov))
fitted1=fit_odeint(t, *popt)

ydata2=np.array(new_Frame['Italy'][i:n])
I0 = ydata2[0]
N0 = 330000000
S0 = N0-I0
popt, pcov = optimize.curve_fit(fit_odeint, t, ydata2) ## Train the model / Fit the model
perr = np.sqrt(np.diag(pcov))
fitted2=fit_odeint(t, *popt)

ydata3=np.array(new_Frame['US'][i:n])
I0 = ydata3[0]
S0 = N0-I0
popt, pcov = optimize.curve_fit(fit_odeint, t, ydata3) ## Train the model / Fit the model
perr = np.sqrt(np.diag(pcov))
fitted3=fit_odeint(t, *popt)


fittedData = pd.DataFrame({'Germany': fitted1,'Italy': fitted2, 'US': fitted3})

ydata = pd.DataFrame({'Germany': ydata1,'Italy': ydata2, 'US': ydata3})

color_list={"Germany": "blue" , "US": "red" , "Italy": "green"}


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data
    This dashboard shows the dynamic SIR-Model of three pre-selected countries
    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='drop_down_country',
        #options=[ {'label': each,'value':each} for each in data_top.unique()],
        options=[ 
            {'label': 'Germany','value': 'Germany'},
            {'label': 'US','value': 'US'},
            {'label': 'Italy','value': 'Italy'}],
        value=['Germany'], # which are pre-selected
        multi=True
    ),

    dcc.Graph(figure=fig, id='main_window_slope1')
])



@app.callback(
    Output(component_id='main_window_slope1', component_property='figure'),
    [Input(component_id='drop_down_country', component_property='value')]
)
def update_figure(country_list):
    
    my_yaxis={'type':"log",
               'title':'Number of Infections'
              }

    traces = []
    for each in country_list:

        #df_plot=df_input_large[df_input_large['country']==each]
        df_plot=ydata
        
        
        
        traces.append(dict(x=t,
                                    y=fittedData[each],
                                    mode='lines',
                                    marker = dict(
                                      size = 3,
                                      color=color_list[each]  
                                    ),
                                    opacity=0.9,
                                    name=each +"fitted"
                            )
                    )
        traces.append(dict(x=t,
                                    y=ydata[each],
                                    mode='markers',
                                    marker = dict(
                                      size = 5,
                                      color=color_list[each]  
                                    ),
                                    opacity=0.9,
                                    name=each
                            )
                    )
        

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                title="SIR Model for 3 countries: Actual data vs fitted model",

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

C:\Users\demb011\Enterprise_Data_Science\Enterprise_Data_Science\notebooks
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on htt